In [98]:
import json
import string
with open('fanficResult.json') as f:
    stories = [x for x in json.load(f)['entities'] if str(x['pageType']) == "story"]
    #print(stories[0]['otherInfo'])
    #table = str.maketrans("", "", string.punctuation)
    #print([int(str(x['otherInfo']['words']).replace(',', '')) for x in stories])
    print(max(stories, key=lambda x: int(str(x['otherInfo']['words']).replace(',', ''))))

{'pageType': 'story', 'storyLink': '/s/6735812/162/The-Spirit-of-Redemption', 'author': '/u/2720337/Myetel', 'title': 'The Spirit of Redemption Chapter 162: Update: Leaves Across the Face of Time, a mass effect fanfic | FanFiction', 'storyType': '/game/Mass-Effect/', 'abstract': 'The Spectres fight against threats to both the galaxy and their families on Mindoir. Shepard/Garrus, multiple OCs, largely expanded universe. Includes Redemption, Hunt, Unity, and Victory.', 'rating': 'Fiction  M', 'otherInfo': {'language': 'English', 'genre': ['Adventure', 'Sci'], 'favorites': '796', 'follows': '453', 'reviews': 0, 'words': '3,404,794', 'chapters': '162'}, 'date': '2018-10-16 08:24', 'characters': {'ron': 100, 'bellatrix': 1, 'tina': 1}}


In [ ]:
()